# Triage Drift Detector Demo

In this notebook, I will illustrate how to use the traige drift detector using a simulated clinical environment.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import random
import wasabi

The key object we want to use is `MultiDriftDetector`.

In [2]:
from triage_detector import triage_detector
from importlib import reload
reload(triage_detector)
MultiDriftDetector = triage_detector.MultiDriftDetector

## Set up the Simulated Environment

We'll use a simple referral template with 10 features. 

I'm still working on compatability with numeric, categorical, and sequential features.

In [43]:
n_features = 10
feature_names = [ f'Feature{i}' for i in range(n_features) ]
feature_names

['Feature0',
 'Feature1',
 'Feature2',
 'Feature3',
 'Feature4',
 'Feature5',
 'Feature6',
 'Feature7',
 'Feature8',
 'Feature9']

And we'll have a label set of 4 priority levels.

In [44]:
n_labels = 4
label_names = [ f'Priority{i}' for i in range(n_labels) ]
label_names

['Priority0', 'Priority1', 'Priority2', 'Priority3']

We'll simulate GPs making referrals by randomly assigning feature values with a probability of `FeatureN` being `True` of 0.2.

In [45]:
class GP:
    feature_rate = 0.2
    def make_referral(self):
        return stats.bernoulli.rvs(p=GP.feature_rate, size=n_features)

gp = GP()
ref = gp.make_referral()
ref

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1])

We'll simulate a model predicting a priority label by giving 90% probability to `Priority0`, and equal probability to the other labels.

In [46]:
class Model:
    only_label = 0
    def predict(self, x):
        prediction = [0.1/(n_labels-1)] * n_labels
        prediction[Model.only_label] = 0.9
        return prediction
    
model = Model()
model.predict(ref)

[0.9, 0.03333333333333333, 0.03333333333333333, 0.03333333333333333]

We'll simulate clinicians labelling the referrals by always assigning `Priority0` to each referral.

In [47]:
# Clinicians always give the lowest priority.
class Clinician:
    only_label = label_names[0]
    def label(self, instance):
        return Clinician.only_label

clinician = Clinician()
clinician.label(ref)

'Priority0'

## Set up the drift detector

The first thing we need to do is specify what should happen when a drift is detected. 

For this demo we'll simply print the message describing what the drift detector has detected.

In [48]:
msg = wasabi.Printer()

def display_message(drift_message):
    msg.info(drift_message)
    
display_message('Somthing interesting has happened.')

ℹ Somthing interesting has happened.


The drift detector writes it's state to a bunch of files in a directory. This allows the dash app to display the history of the detector, and the detector to be restored if its session is interrupted. The directory we will write to is:

In [49]:
write_dir = './data/demo'

We can now instantiate the detector.

In [50]:
# Instantiate a TraigeDetector object
detector = MultiDriftDetector(
    write_dir = write_dir,
    drift_action = display_message
)

We need to let the detector know what are the features called, and what are the possible labels.

In [51]:
# Specify the features
detector.set_features(feature_names)

# Specify the labels
detector.set_labels(label_names)

## Begin the Simulation

We need to give each referral an ID so that documents, predictions, and labels can be matched up. For simplicity, we'll give the $n$-th created document an id of $n$. We'll use a variable `id_count` to keep track of what the next id should be.

All the referral documents which haven't yet been given a label by a clinician will be stored up in a list called `backlog`.

In [52]:
id_count = 0
backlog = []

We'll create a function `new_referral()` to simulate the following actions:
 * a GP makes a new referral
 * the model predicts the priority label for this referral
 * the new referral document and prediction are registered with the drift detector

In [53]:
def new_referral():    
    global backlog, id_count
    
    # A GP makes a new referral and the model predicts a triage label.
    ref = gp.make_referral()
    pred = model.predict(ref)

    # We need to register the new referral and prediction with the drift detector.
    detector.add_instance(ref, instance_id=id_count, description=f'ID={id_count}')
    detector.add_prediction(pred, instance_id=id_count, description=f'ID={id_count}')

    # Print an update
    print(f'A GP has created a referral with id {id_count}.')

    # Add the new referral to the backlog and update the id counter
    backlog.append( (id_count, ref) )
    id_count += 1

We'll start the simulation with an initial backlog of 20 referral documents.

In [54]:
for i in range(20):
    new_referral()

A GP has created a referral with id 0.
A GP has created a referral with id 1.
A GP has created a referral with id 2.
A GP has created a referral with id 3.
A GP has created a referral with id 4.
A GP has created a referral with id 5.
A GP has created a referral with id 6.
A GP has created a referral with id 7.
A GP has created a referral with id 8.
A GP has created a referral with id 9.
A GP has created a referral with id 10.
A GP has created a referral with id 11.
A GP has created a referral with id 12.
A GP has created a referral with id 13.
A GP has created a referral with id 14.
A GP has created a referral with id 15.
A GP has created a referral with id 16.
A GP has created a referral with id 17.
A GP has created a referral with id 18.
A GP has created a referral with id 19.


We'll use a function `new_label()` to simulate the following steps:
 * a clinician takes a referral from the backlog and labels it
 * the new label is registered with the drift detector

In [55]:
def new_label():
    global backlog
    
    # If the backlog is empty then do nothing
    if len(backlog)==0:
        return
    
    # Randomly choose a referral and remove it from the backlog
    i = random.randrange(len(backlog))
    (ref_id, ref) = backlog[i]
    del(backlog[i])

    # Label the referral and register it
    label = clinician.label(ref)
    detector.add_label(label, instance_id=ref_id, description=f'ID={ref_id}')

    # Print an update
    print(f'A clincian has labelled a referral with id {ref_id}.')
    
new_label()

A clincian has labelled a referral with id 6.


Let's now simulate 100 timesteps where either a new referral is sent or a clinician labels a document (with equal probability).

In [56]:
for t in range(100):
    if random.random() < 0.5:
        new_referral()
    else:
        new_label()

A GP has created a referral with id 20.
A GP has created a referral with id 21.
A GP has created a referral with id 22.
A GP has created a referral with id 23.
A GP has created a referral with id 24.
A GP has created a referral with id 25.
A clincian has labelled a referral with id 20.
A GP has created a referral with id 26.
A clincian has labelled a referral with id 22.
A GP has created a referral with id 27.
A clincian has labelled a referral with id 9.
A clincian has labelled a referral with id 13.
A clincian has labelled a referral with id 25.
A clincian has labelled a referral with id 27.
A clincian has labelled a referral with id 11.
A clincian has labelled a referral with id 15.
A GP has created a referral with id 28.
A GP has created a referral with id 29.
A GP has created a referral with id 30.
A clincian has labelled a referral with id 14.
A GP has created a referral with id 31.
A GP has created a referral with id 32.
A GP has created a referral with id 33.
A clincian has lab

Let's look at the status of the detector.

In [57]:
detector.get_status()

✔ Loss distribution normal.
✔ Feature distribution normal.
✔ Label distribution normal.


Nothing detected so far. I'll describe what each of these lines means further on in this notebook.

## Feature Drift

We now simulate a "feature drift". That is, a change in the distribution of the features $P(x)$.

In the clinical setting, this may indicate:
 * the demographics of the population have changed
 * some condition has increased/decreased in the population
 
A feature drift may or may not require model retraining. On the one hand, the decision boundary won't change. On the other hand, the accuracy of the model *can* change.
 
We'll simulate feature drift by increasing the rate that clinicians assign positive values to features.

In [58]:
GP.feature_rate = 0.6

Let's run another 100 timesteps.

In [59]:
for t in range(100):
    if random.random() < 0.5:
        new_referral()
    else:
        new_label()

A clincian has labelled a referral with id 77.
A GP has created a referral with id 78.
A clincian has labelled a referral with id 71.
A GP has created a referral with id 79.
A GP has created a referral with id 80.
A GP has created a referral with id 81.
A clincian has labelled a referral with id 72.
A GP has created a referral with id 82.
ℹ The status of Feature4 has changed to
DRIFT
ℹ The status of Feature8 has changed to
WARNING
ℹ The status of Feature9 has changed to
DRIFT
A GP has created a referral with id 83.
A clincian has labelled a referral with id 66.
ℹ The status of Feature8 has changed to
DRIFT
A GP has created a referral with id 84.
A GP has created a referral with id 85.
A clincian has labelled a referral with id 63.
A GP has created a referral with id 86.
A GP has created a referral with id 87.
A clincian has labelled a referral with id 79.
ℹ The status of Feature0 has changed to
WARNING
A GP has created a referral with id 88.
A clincian has labelled a referral with id 7

We see that the detector has sent several messages describing changes in the distribution of the features.

Note that a `WARNING` signal is sent when drift is suspected but not yet confirmed.

In [60]:
detector.get_status()

✔ Loss distribution normal.
✘ Feature drift detected on the following: Feature0, Feature1,
Feature2, Feature3, Feature4, Feature6, Feature7, Feature8, Feature9
✔ Label distribution normal.


The detector has detected drift for all of the features except `Feature5`.

## Concept Drift

We'll now simulate concept drift, that is a change in the distribution $P(y|x)$. In this demo, $y$ and $x$ are indepedent, so we need only change $P(y)$. A change in $P(y)$ is label drift, which MultiDriftDetector also detects.

We'll simulate concept drift (and label drift) by changing the label which the model assigns 90% probability from `Priority0` to `Priority1`.

In [61]:
Model.only_label = 1
model.predict(None)

[0.03333333333333333, 0.9, 0.03333333333333333, 0.03333333333333333]

We now run another 100 timesteps.

In [62]:
for t in range(100):
    if random.random() < 0.5:
        new_referral()
    else:
        new_label()

A clincian has labelled a referral with id 126.
A GP has created a referral with id 127.
ℹ The status of Feature5 has changed to
DRIFT
A GP has created a referral with id 128.
A GP has created a referral with id 129.
A clincian has labelled a referral with id 12.
A clincian has labelled a referral with id 124.
A clincian has labelled a referral with id 87.
ℹ The status of Priority0 has changed to
WARNING
ℹ The status of Priority1 has changed to
WARNING
A GP has created a referral with id 130.
A clincian has labelled a referral with id 125.
A clincian has labelled a referral with id 128.
ℹ The status of Priority0 has changed to
DRIFT
ℹ The status of Priority1 has changed to
DRIFT
A GP has created a referral with id 131.
A clincian has labelled a referral with id 131.
A clincian has labelled a referral with id 102.
A clincian has labelled a referral with id 97.
A GP has created a referral with id 132.
A clincian has labelled a referral with id 121.
A clincian has labelled a referral with

In [42]:
detector.get_status()

✘ Concept drift detected.
✘ Feature drift detected on the following: Feature0, Feature1,
Feature2, Feature3, Feature4, Feature6, Feature8, Feature9
✘ Label drift detected on the following: Priority0, Priority1


First, the drift in `Feature5` was detected. 

Next, changes in the distribution of `Priority0` and `Priority1` were detected, due to the changes in the distribution of the predictions.

Finally, concept drift was detecting, due to a change in the error rate. Rather than directly detecting changes in $P(y|x)$, which is very hard, drift detectors instead look for an increase in $P(\hat{y}=y)$, where $\hat{y}$ is the predicted label and $P(y)$ is the true label.